In [1]:
import numpy as np
import sys  
import matplotlib.pyplot as plt
import pickle
import gymnasium
import mountain_lite
fd = open('trajectories.pickle','rb')
D = pickle.load(fd)

np.random.seed(10)

env = gymnasium.make('mountain_lite/GridWorld-v0')

FileNotFoundError: [Errno 2] No such file or directory: 'trajectories.pickle'

In [11]:
len(D)

1000

In [12]:
len(D[0])

500

In [13]:
len(D[0][0])

4

In [26]:
D[0][0:9]

[((15, 0), 0, -0.5058824, (14, 0)),
 ((14, 0), 6, -0.5058824, (14, 0)),
 ((14, 0), 5, -0.5058824, (15, 1)),
 ((15, 1), 3, -0.5058824, (16, 0)),
 ((16, 0), 3, -0.5058824, (17, 0)),
 ((17, 0), 6, -0.5058824, (17, 0)),
 ((17, 0), 0, -0.5058824, (16, 0)),
 ((16, 0), 5, -0.5058824, (17, 0)),
 ((17, 0), 1, -0.5058824, (16, 0))]

In [27]:
D[0][100:109]

[((21, 7), 6, -0.2, (21, 6)),
 ((21, 6), 1, -0.5058824, (20, 7)),
 ((20, 7), 6, -0.41960785, (20, 6)),
 ((20, 6), 0, -0.5058824, (19, 5)),
 ((19, 5), 6, -0.5882353, (19, 4)),
 ((19, 4), 6, -0.6509804, (19, 3)),
 ((19, 3), 6, -0.627451, (19, 2)),
 ((19, 2), 5, -0.5411765, (20, 3)),
 ((20, 3), 3, -0.5058824, (21, 2))]

In [ ]:
def epsilon_greedy_policy(Q, state, epsilon):
    if np.random.randint() < epsilon:
        return np.random.randint(Q.shape[2])
    else:
        return np.argmax(Q[state])


def update_SARSA_n(Q, state_action_pairs, rewards, alpha, gamma, n):
    # Calculate cumulative reward with decay
    G = 0
    for reward in rewards:
        G += reward*gamma
        gamma *= gamma
    # G = sum([gamma ** i * rewards[i] for i in range(len(rewards))])
    # Calculate the final state and action
    final_state, final_action = state_action_pairs[-1]
    # Update Q value for the starting state and action
    start_state, start_action = state_action_pairs[0]
    Q[start_state][start_action] += alpha * (G + (gamma ** n) * Q[final_state][final_action] - Q[start_state][start_action])
    return Q


def SARSA_n(env, episodes, alpha, gamma, epsilon, n, random_start=True):
    shape = (env.height, env.width, 8)  
    Q = np.zeros(shape)

    for episode in tqdm(range(episodes)):
        epsilon *= 0.999
        # Initialization for each episode
        state_action_pairs = []
        rewards = []
        if random_start and episode < 9 * episodes // 10:
            start_col = int(env.width-1-episode*env.width//episodes)
            env.set_start_position((np.random.randint(env.height),start_col))
        else:
            env.set_start_position((15, 0))
        # Reset environment and get initial state and action
        observation, info = env.reset(seed=episode)
        state = tuple(observation["agent"]["pos"])
        action = epsilon_greedy_policy(Q, state, epsilon)
        state_action_pairs.append((state, action))

        done = False
        while not done:
            observation, reward, terminated, truncated, info = env.step(action)
            next_state = tuple(observation["agent"]["pos"])
            next_action = epsilon_greedy_policy(Q, next_state, epsilon)

            # Store state, action, and reward
            state_action_pairs.append((next_state, next_action))
            rewards.append(reward)

            if len(rewards) >= n or terminated or truncated:
                Q = update_SARSA_n(Q, state_action_pairs, rewards, alpha, gamma, n)
                # Reset the state_action_pairs and rewards for the next update
                state_action_pairs = state_action_pairs[-1:]
                rewards = []

            state, action = next_state, next_action
            done = terminated or truncated

    return Q

In [ ]:

def epsilon_greedy_policy(Q, state, epsilon):
    # Correct the probability check and action selection in the epsilon-greedy policy
    if np.random.rand() < epsilon:
        return np.random.randint(Q.shape[2])
    else:
        return np.argmax(Q[state])

def sarsa(env, episodes, alpha, gamma, epsilon):
    # Initialize Q-values based on the environment's dimensions for dynamic compatibility
    shape = (env.height, env.width, 8)  # Assuming 8 possible actions
    Q = np.zeros(shape)  # Adjusted to use the 'shape' variable
    
    for episode in range(episodes):
        state = env.reset()[0]['agent']['pos']  # Start a new episode and get the initial state
        state = tuple(state)
        action = epsilon_greedy_policy(Q, state, epsilon)
      
        for _ in range(500):  # Limit the number of steps in each episode
            next_state, reward, done, _, _ = env.step(action)  # Take action
            next_state = tuple(next_state['agent']['pos'])
            
            next_action = epsilon_greedy_policy(Q, next_state, epsilon)  # Choose next action
            
            # SARSA(0) update rule
            Q[state][action] += alpha * (reward + gamma * Q[next_state][next_action] - Q[state][action])
            
            state, action = next_state, next_action  # Move to the next state and action
            
            if done:
                break
    
    # Derive policy from Q-values
    policy = np.argmax(Q, axis=2)
    return Q, policy

# Parameters (example, adjust as necessary)
env = None  # Placeholder for your environment
alpha = 0.1  # Learning rate
gamma = 0.9  # Discount factor
epsilon = 0.1  # Exploration rate
episodes = 1000  # Number of episodes

# Assuming 'env' is properly defined elsewhere
# Q, policy = sarsa(env, episodes, alpha, gamma, epsilon)
